# N-Gram Model Implementation

**Definition**  
An n-gram is a sequence of *n* consecutive tokens (words).  
The model approximates the probability of the next token as:  

$$
P(w_t \mid w_{t-n+1}^{t-1}) \approx 
\frac{\text{count}(w_{t-n+1}, \dots, w_{t})}
     {\text{count}(w_{t-n+1}, \dots, w_{t-1})}
$$

**Example (n=1,2,3)**  
Sentence: `"i love machine learning"`  
- unigram (*n=1*): `i` → predicts `love`  
- bigram (*n=2*): `i love` → predicts `machine`  
- trigram (*n=3*): `i love machine` → predicts `learning`  

**Estimation**  
We predict the next token by estimating its probability from counts in the training data:  

$$
P(\text{learning} \mid \text{machine}) = 
\frac{\text{count}(\text{machine, learning})}{\text{count}(\text{machine})}
$$

**Corpus example**  
["i love machine learning", "machine learning is awesome", "this machine is awesome"]

- counts:  
  - `("machine", "learning") = 2`  
  - `("machine", "is") = 1`  
  - `("machine") = 3`  

$$
P(\text{learning} \mid \text{machine}) = \tfrac{2}{3}, \quad
P(\text{is} \mid \text{machine}) = \tfrac{1}{3}
$$

Therefore, the model predicts **`learning`** after `"machine"`.


### Step 1 - Tokenize

Before building n-grams, we must split text into tokens (usually words).  

**Example**: `"The cat sat."` → `["the", "cat", "sat"]`  

To implement this we use regex:
- `\b`: start or end of a word
- `\w`: one word character [a-z] [A-Z] [0-9] [_]
- `\w+`: multiple word characters

We also normalize (e.g., lowercase) so counts are consistent.

In [8]:
from typing import List
import re

def tokenize(text:str) -> List[str]:
    tokens = re.findall(r"\b\w+\b", text.lower())
    return tokens

print(tokenize("I love machine learning!"))

['i', 'love', 'machine', 'learning']


### Step 2 - Build Vocabulary

The vocabulary is the set of tokens the model knows. We build the vocab by collecting all tokens from the training data. We also add a `unk` token to handle unseen tokens later

In [11]:
from typing import Set, Tuple

def build_vocab(texts: List[str]) -> Set[str]:
    vocab = {"<unk>"}
    for text in texts:
        vocab.update(tokenize(text))
    return vocab

corpus = ["I love machine learning.", "Machine learning is Awesome!", "This machine is awesome."]

vocab = build_vocab(corpus)
print(vocab)

{'this', 'learning', '<unk>', 'is', 'i', 'machine', 'love', 'awesome'}


### Step 3 - Map unkown tokens

When we apply the model to new text, we may see tokens that are not in our vocab. To handle this, we replace them with the `unk` token.

In [20]:
def map_unk(tokens: List[str], vocab: Set[str]) -> List[str]:
    mapped_tokens = []
    for token in tokens:
        if token not in vocab: 
            mapped_tokens.append("<unk>")
        else:
            mapped_tokens.append(token)
    return mapped_tokens

text_new_vocab = "I love icecream."
tokens = tokenize(text_new_vocab)
print(map_unk(tokens, vocab))

['i', 'love', '<unk>']


### Step 4 - Count

To keep things simple in this exercise, we will implement a bigram (*n=2*) model here. Therefor, we have to count all single tokens and pairs in our training data. 

We approximate the probability of a token following a given token as follows: 

$$
P(w_t \mid w_{t-1}) \approx 
\frac{\text{count}(w_{t-1}, w_{t})}
     {\text{count}(w_{t-1})}
$$

In [22]:
from collections import Counter
from typing import Tuple

def text_pipeline(text: str, vocab: Set[str]) -> List[str]:
    return map_unk(tokenize(text), vocab)

def count_tokens(corpus: List[str], vocab: Set[str]) -> Counter:
    counts = Counter()
    for text in corpus:
        for token in text_pipeline(text, vocab):
            counts[token] += 1
    return counts

def count_pairs(corpus: List[str], vocab: Set[str]) -> Counter:
    pairs = Counter()
    for text in corpus:
        tokens = text_pipeline(text, vocab)
        for i in range(1, len(tokens)):
            prev, curr = tokens[i-1], tokens[i]
            pairs[(prev, curr)] += 1
    return pairs

print(count_tokens(corpus, vocab))
print(count_pairs(corpus,vocab))

Counter({'machine': 3, 'learning': 2, 'is': 2, 'awesome': 2, 'i': 1, 'love': 1, 'this': 1})
Counter({('machine', 'learning'): 2, ('is', 'awesome'): 2, ('i', 'love'): 1, ('love', 'machine'): 1, ('learning', 'is'): 1, ('this', 'machine'): 1, ('machine', 'is'): 1})


### Step 5 - Predicts

Great, now we have everything set up to make our prediction. 

In [25]:
def predict_next(prev: str, vocab: Set[str],
                 tokens: Counter, pairs: Counter) -> str | None:
    # find all followers of prev
    followers = [w for (p, w) in pairs if p == prev]
    if not followers:
        return None

    prev_count = tokens[prev]  # denominator

    # compute probabilities
    probs = {w: pairs[(prev, w)] / prev_count for w in followers}

    # return the follower with highest probability
    return max(probs, key=probs.get)

vocab = build_vocab(corpus)
tokens = count_tokens(corpus, vocab)
pairs  = count_pairs(corpus, vocab)

print(predict_next("machine", vocab, tokens, pairs)) 

learning
